In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
%config InlineBackend.figure_format = 'png'

matplotlib.rcParams['pdf.fonttype'] = 42

In [ ]:
# Parse output of end-to-end benchmarks
class MageMeasurement(object):
    def __init__(self, f):
        lines = []
        for line in f:
            tokens = line.split()
            if len(tokens) >= 2 and tokens[1] == "(ns)":
                stats = Stats(line)
                if stats.name not in self.stats:
                    self.stats[stats.name] = []
                self.stats[stats.name].append(stats.total)
            elif len(tokens) == 3 and tokens[0] == "Timer:" and tokens[2] == "ns":
                self.time_for_computation_ns = int(tokens[1])
                self.time_for_computation_ms = self.time_for_computation_ns / 1000000.0
            lines.append(line)
        
        tokens = lines[-1].split()
        if len(tokens) == 2 and tokens[1] == "ms":
            self.total_time_ms = int(tokens[0])
        else:
            assert(False)
            self.total_time_ms = self.time_for_computation_ns / 1000000.0 # Hack to produce graphs with some missing data

class EMPMeasurement(object):
    def __init__(self, f):
        lines = []
        for line in f:
            lines.append(line)
        assert(len(lines) == 4)
        assert(lines[0] == "connected\n")
        assert(lines[2] == "PASS\n")
        self.time_for_computation_ms = int(lines[1].split()[1])
        self.total_time_ms = int(lines[3].split()[1])
        
class PlanningMeasurement(object):
    def __init__(self, f):
        lines = []
        for line in f:
            lines.append(line)
        assert(len(lines) == 6)
        phase_times = lines[5].split()
        self.placement_ms = int(phase_times[3])
        self.replacement_ms = int(phase_times[4])
        self.scheduling_ms = int(phase_times[5])
        self.total_ms = self.placement_ms + self.replacement_ms + self.scheduling_ms
        
class SEALMeasurement(object):
    def __init__(self, f):
        lines = []
        for line in f:
            lines.append(line)
        assert(len(lines) == 1)
        self.total_time_ms = int(lines[0].split()[0])

class Stats(object):
    def __init__(self, line):
        tokens = line.strip().split()
        self.name = tokens[0]
        self.unit = tokens[1][1:-2]
        assert(tokens[3] == "min")
        assert(tokens[6] == "avg")
        assert(tokens[9] == "max")
        assert(tokens[12] == "count")
        assert(tokens[15] == "sum")
        self.total = int(tokens[17])
        
def parse_planning_log(f):
    last_line = None
    for line in f:
        last_line = line
    tokens = last_line.split()
    return tuple(int(x) for x in tokens[3:])

In [ ]:
def generate_filename(prefix, program, scenario, tag):
    if isinstance(tag, int):
        tag = "t{0}".format(tag)
    return "{0}_{1}_{2}_{3}.log".format(prefix, program, scenario, tag)

def parse_emp_measurement_file(filename):
    with open(filename) as f:
        return EMPMeasurement(f)

def parse_mage_measurement_file(filename):
    with open(filename) as f:
        return MageMeasurement(f)
    
def parse_seal_measurement_file(filename):
    with open(filename) as f:
        return SEALMeasurement(f)
    
def parse_planning_measurement_file(filename):
    with open(filename) as f:
        return PlanningMeasurement(f)
    
def parse_planning_log_file(filename):
    with open(filename) as f:
        return parse_planning_log(f)
    
def parse_log_directory(directory):
    logs = {}
    machine_directories = !ls {directory}
    for mdir in machine_directories:
        try:
            machine_id = int(mdir)
        except ValueError:
            print("Skipping directory {0}", os.path.join("directory", mdir))
            continue
            
        machine_logs = logs.setdefault(machine_id, {})
            
        log_files = !ls {os.path.join(directory, mdir)}
        for log_file in log_files:
            if log_file.endswith(".log") or log_file.endswith(".planning"):
                log_path = os.path.join(directory, mdir, log_file)
                
                parts = log_file.split(".")
                extension = parts[-1]
                name = ".".join(parts[:-1])
                
                ext_logs = machine_logs.setdefault(extension, {})
                
                tokens = name.split("_")
                if len(tokens) < 6:
                    print("Skipping file {0}".format(log_path))
                    continue
                experiment = "_".join(tokens[:2])
                scenario = tokens[-2]
                try:
                    size = int(tokens[-3])
                except ValueError:
                    print("Skipping file {0}".format(log_path))
                    continue
                problem = "_".join(tokens[2:-3])
                
                tag = tokens[-1]
                
                experiments = ext_logs.setdefault(experiment, {}).setdefault(problem, {}).setdefault(size, {}).setdefault(scenario, {})
                if tag in experiments:
                    print("Skipping {0} (duplicate for {1})".format(log_path, (experiment, problem, size, scenario, tag)))
                    continue
                    
                try:
                    if scenario in ("os", "unbounded", "mage"):
                        if extension == "log":
                            parsed = parse_mage_measurement_file(log_path)
                        elif extension == "planning":
                            parsed = parse_planning_measurement_file(log_path)
                    elif scenario == "emp":
                        parsed = parse_emp_measurement_file(log_path)
                    elif scenario == "seal":
                        parsed = parse_seal_measurement_file(log_path)
                    else:
                        print("Skipping {0} (unknown scenario {1})".format(log_path, scenario))
                        continue
                    experiments[tag] = parsed
                except AssertionError as ae:
                    print("Skipping {0} (assertion failure: {1})".format(log_path, str(ae)))
    return logs

In [ ]:
cluster2_logs = parse_log_directory("logs-2")
cluster8_logs = parse_log_directory("logs")

Baseline Experiments (Figures 6 and 7)
================================

In [ ]:
baseline_unbounded_style = {"label": "Unbounded", "color": "blue", "marker": "s", "ls": "-", "linewidth": 3}       
baseline_mage_style = {"label": "MAGE 1 GiB", "color": "orange", "marker": "^", "ls": "--"}
baseline_os_style = {"label": "OS 1 GiB", "color": "green", "marker": "o", "ls": "-"}
baseline_emp_style = {"label": "EMP 1 GiB", "color": "red", "marker": "v", "ls": "-."}
baseline_seal_style = {"label": "SEAL 1 GiB", "color": "red", "marker": "v", "ls": "-."}

In [ ]:
def draw_halfgates_baseline_graph(sizes):
    plt.figure(figsize = (3, 2))
    
    benchmark_data = cluster2_logs[0]["log"]["halfgates_baseline"]["merge_sorted"]
    def get_data(scenario, sizes, tag):
        result = []
        for size in sizes:
            time_s = benchmark_data[size][scenario][tag].time_for_computation_ms / 1000.0
            result.append(time_s)
        return result

    plt.plot(sizes, get_data("unbounded", sizes, "t1"), **baseline_unbounded_style)
    plt.plot(sizes, get_data("os", sizes, "t1"), **baseline_os_style)
    plt.plot(sizes, get_data("mage", sizes, "t1"), **baseline_mage_style)
    plt.plot(sizes, get_data("emp", sizes, "t1"), **baseline_emp_style)

    plt.xlim(0, sizes[-1])
    
    plt.xlabel("Problem Size (Records Per Party)")
    plt.ylabel("Time (s)")
    
def draw_ckks_baseline_graph(sizes):
    plt.figure(figsize = (3, 2))
    
    benchmark_data = cluster2_logs[0]["log"]["ckks_baseline"]["real_statistics"]
    def get_data(scenario, sizes, tag):
        result = []
        for size in sizes:
            time_s = benchmark_data[size][scenario][tag].total_time_ms / 1000.0
            result.append(time_s)
        return result

    plt.plot(sizes, get_data("unbounded", sizes, "t1"), **baseline_unbounded_style)
    plt.plot(sizes, get_data("os", sizes, "t1"), **baseline_os_style)
    plt.plot(sizes, get_data("mage", sizes, "t1"), **baseline_mage_style)
    plt.plot(sizes, get_data("seal", sizes, "t1"), **baseline_seal_style)

    plt.xlim(0, sizes[-1])
    
    plt.xlabel("Problem Size (Number of Elements)")
    plt.ylabel("Time (s)")

In [ ]:
draw_halfgates_baseline_graph(tuple(2 ** i for i in range(13, 20)))
plt.ylim(0, 200)

plt.legend(loc=2)
plt.show()

In [ ]:
draw_halfgates_baseline_graph(tuple(2 ** i for i in range(13, 21)))
plt.ylim(0, 1000)

plt.xticks((0, 500000, 1000000))
plt.ticklabel_format(style = "plain")

plt.legend(loc=2)
plt.show()

In [ ]:
draw_ckks_baseline_graph(tuple(2 ** i for i in range(8, 13)))
plt.ylim(0, 10)

plt.legend()
plt.show()

In [ ]:
draw_ckks_baseline_graph(tuple(2 ** i for i in range(8, 15)))
plt.ylim(0, 250)

plt.legend()
plt.show()

Single-Node Experiments
=====================

In [ ]:
barwidth = 0.2
errbarsize = 3
styles = [{"label": "Unbounded", \
            "width": barwidth, "capsize": errbarsize, "color": "blue", "hatch": "", "edgecolor": "black"},          
          {"label": "MAGE 1 GiB", \
           "width": barwidth, "capsize": errbarsize, "color": "white", "hatch": "..", "edgecolor": "black"},
          {"label": "OS 1 GiB", \
           "width": barwidth, "capsize": errbarsize, "color": "white", "hatch": "\\\\", "edgecolor": "black"}]

In [ ]:
def draw_workloads_plot(experiments, graph_data, experiment_display_names):
    x = np.arange(0, len(experiments))
    unbounded_err = [[(graph_data[e]["unbounded"][2] - graph_data[e]["unbounded"][1]) / graph_data[e]["unbounded"][2] for e in experiments], [(graph_data[e]["unbounded"][3] - graph_data[e]["unbounded"][2]) / graph_data[e]["unbounded"][2] for e in experiments]]
    plt.bar(x - barwidth, [graph_data[e]["unbounded"][2] / graph_data[e]["unbounded"][2] for e in experiments], yerr = unbounded_err, **styles[0])
    mage_err = [[(graph_data[e]["mage"][2] - graph_data[e]["mage"][1]) / graph_data[e]["unbounded"][2] for e in experiments], [(graph_data[e]["mage"][3] - graph_data[e]["mage"][2]) / graph_data[e]["unbounded"][2] for e in experiments]]
    plt.bar(x, [graph_data[e]["mage"][2] / graph_data[e]["unbounded"][2] for e in experiments], yerr = mage_err, **styles[1])
    os_err = [[(graph_data[e]["os"][2] - graph_data[e]["os"][1]) / graph_data[e]["unbounded"][2] for e in experiments], [(graph_data[e]["os"][3] - graph_data[e]["os"][2]) / graph_data[e]["unbounded"][2] for e in experiments]]
    plt.bar(x + barwidth, [graph_data[e]["os"][2] / graph_data[e]["unbounded"][2] for e in experiments], yerr = os_err, **styles[2])
    
    plt.xticks(x, experiment_display_names)
    plt.ylabel("Time (Normalized\nby Unbounded)")

In [ ]:
hg_experiments = (("merge_sorted", 1048576), ("full_sort", 1048576), ("loop_join", 2048), ("matrix_vector_multiply", 8192), ("binary_fc_layer", 16384))
ckks_experiments = (("real_sum", 65536), ("real_statistics", 16384), ("real_matrix_vector_multiply", 256), ("real_naive_matrix_multiply", 128), ("real_tiled_matrix_multiply", 128))

experiments = hg_experiments + ckks_experiments
experiment_display_names = ("merge\nn = 1048576", "sort\nn = 1048576", "ljoin\nn = 2048", "mvmul\nn = 8192", "binfclayer\nn = 16384", "rsum\nn = 65536", "rstats\nn = 16384", "rmvmul\nn = 256", "n_rmatmul\nn = 128", "t_rmatmul\nn = 128")

In [ ]:
plt.figure(figsize = (15, 2))

# TODO: Support multiple trials done in parallel across different pairs of machines
benchmark_data = cluster2_logs[0]["log"]["single_machine"]
graph_data = {}
for i, e in enumerate(experiments):
    graph_data[e] = {}
    print(experiment_display_names[i].split("\n")[0], end = "")
    for scenario in ("unbounded", "os", "mage"):
        total_times = tuple(m.total_time_ms / 1000.0 for tag, m in benchmark_data[e[0]][e[1]][scenario].items())
        assert len(total_times) > 0
        graph_data[e][scenario] = np.percentile(total_times, (0, 25, 50, 75, 100))
        print(" &", round(np.median(total_times), 1), end="")
    print(" &", round(graph_data[e]["os"][2] / graph_data[e]["mage"][2], 1), end="")
    print(" &", round((graph_data[e]["mage"][2] / graph_data[e]["unbounded"][2] - 1) * 100, 0), end="\\%")
    print()

draw_workloads_plot(experiments, graph_data, experiment_display_names)

# Uncomment this line for the scale to match the one in the figure in the paper.
# plt.ylim(0, 11)

plt.legend()
plt.show()

In [ ]:
# TODO: Support multiple trials done in parallel across different pairs of machines
planning_data = cluster2_logs[0]["planning"]["single_machine"]
for i, e in enumerate(experiments):
    graph_data[e] = {}
    print(experiment_display_names[i].split("\n")[0], end = "")
    planning_times = tuple(m.total_ms / 1000.0 for tag, m in planning_data[e[0]][e[1]]["mage"].items())
    assert len(planning_times) > 0
    graph_data[e][scenario] = np.percentile(planning_times, (0, 25, 50, 75, 100))
    median = np.median(planning_times)
    if median < 0.1:
        rounded_median = round(median, 3)
    elif median < 1:
        rounded_median = round(median, 2)
    else:
        rounded_median = round(median, 1)
    print(" &", rounded_median, end="\n")

p = 4 Parallelism Experiments
========================

In [ ]:
hg_experiments_4 = (("merge_sorted", 4194304), ("full_sort", 4194304), ("loop_join", 4096), ("matrix_vector_multiply", 16384), ("binary_fc_layer", 32768))
ckks_experiments_4 = (("real_sum", 262144), ("real_statistics", 65536), ("real_matrix_vector_multiply", 512), ("real_naive_matrix_multiply", 256), ("real_tiled_matrix_multiply", 256))

experiments_4 = hg_experiments_4 + ckks_experiments_4
experiment_4_display_names = ("merge\nn = 4194384", "sort\nn = 4194384", "ljoin\nn = 4096", "mvmul\nn = 16384", "binfclayer\nn = 32768", "rsum\nn = 262144", "rstats\nn = 32768", "rmvmul\nn = 512", "n_rmatmul\nn = 256", "t_rmatmul\nn = 256")

In [ ]:
plt.figure(figsize = (15, 2))

# TODO: Support multiple trials done in parallel across different pairs of machines
benchmark_data = cluster2_logs[0]["log"]["single_machine"]
graph_data = {}
for i, e in enumerate(experiments_4):
    graph_data[e] = {}
    print(experiment_4_display_names[i].split("\n")[0], end = "")
    for scenario in ("unbounded", "os", "mage"):
        instances_per_tag = {}
        for machine in (0, 1, 2, 3):
            machine_trials = cluster8_logs[machine]["log"]["workers_4"][e[0]][e[1]][scenario]
            for tag, m in machine_trials.items():
                instances_per_tag.setdefault(tag, []).append(m)
                
        total_times = []
        for tag, machine_exps in instances_per_tag.items():
            assert len(machine_exps) == 4
            total_times.append(max(m.total_time_ms / 1000.0 for m in machine_exps))
        assert len(total_times) > 0
        graph_data[e][scenario] = np.percentile(total_times, (0, 25, 50, 75, 100))
        print(" &", round(np.median(total_times), 1), end="")
    print(" &", round(graph_data[e]["os"][2] / graph_data[e]["mage"][2], 1), end="")
    print(" &", round((graph_data[e]["mage"][2] / graph_data[e]["unbounded"][2] - 1) * 100, 0), end="\\%")
    print()

draw_workloads_plot(experiments_4, graph_data, experiment_4_display_names)

# Uncomment this line for the scale to match the one in the figure in the paper.
# plt.ylim(0, 11)

plt.legend()
plt.show()